<a href="https://colab.research.google.com/github/sandushiw98/Early-Detection-of-Dysgraphia-in-Sinhala-Speaking-Children-Using-Multi-Modal-Machine-Learning/blob/main/Integration_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Labeling the Handwriting Data

In [ ]:
import numpy as np
import cv2

# Preprocess the specific image
image_path = '/content/drive/MyDrive/Research_Dysgraphia/Dysgraphia_Handwritings -Before_Preprocessing/1-daruwa.png'

def preprocess_image(image_path, target_size=(256, 256)):
    # Load image using OpenCV in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found at the given path.")

    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply adaptive thresholding with refined parameters
    thresh_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 9, 4)

    # Morphological erosion to clean up the image
    kernel_erode = np.ones((2, 2), np.uint8)
    morph_image = cv2.morphologyEx(thresh_image, cv2.MORPH_ERODE, kernel_erode)

    # Apply a slight dilation to enhance the characters
    kernel_dilate = np.ones((3, 3), np.uint8)
    morph_image = cv2.morphologyEx(morph_image, cv2.MORPH_DILATE, kernel_dilate)

    # Resize the processed image
    resized_image = cv2.resize(morph_image, target_size)

    # Normalize the image
    normalized_image = resized_image / 255.0

    # Add batch dimension
    final_image = np.expand_dims(normalized_image, axis=0)

    return final_image

# Preprocess the image
preprocessed_image = preprocess_image(image_path)
print(f"Preprocessed image shape: {preprocessed_image.shape}")

Preprocessed image shape: (1, 256, 256)


# Train the model with CNN


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/drive/MyDrive/Research_Dysgraphia/New_VGG16_model_fold_4.h5')  # Update the path to your model

# Function to preprocess the input image
def preprocess_image(image_path, target_size=(128, 128)):
    # Load image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found at the given path.")

    # Resize the image
    image = cv2.resize(image, target_size)

    # Convert image to RGB (since OpenCV loads it as BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Normalize the image
    image = image / 255.0

    # Add batch dimension
    image = np.expand_dims(image, axis=0)

    return image

# Predict function for a single image
def predict_image(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)

    # Get the prediction
    prediction = model.predict(preprocessed_image)

    # Since the model is using a sigmoid activation function, output is between 0 and 1
    if prediction[0][0] > 0.5:
        print("Dysgraphic, please proceed to check your severity.")
        clinical_data_path = input("Enter the path to your clinical data CSV file: ")
        print(f"Clinical data path: {clinical_data_path}")
        # You can add further code here to process the CSV file if needed
    else:
        print("Non-Dysgraphic")

# Example usage
image_path = '/content/drive/MyDrive/Research_Dysgraphia/Dysgraphia_Handwritings -Before_Preprocessing/1-daruwa.png'
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
Dysgraphic, please proceed to check your severity.
Enter the path to your clinical data CSV file: /content/drive/MyDrive/Research_Dysgraphia/ANN-Unseen data.csv
Clinical data path: /content/drive/MyDrive/Research_Dysgraphia/ANN-Unseen data.csv


In [ ]:
import pandas as pd
import joblib

# Load the normalized clinical data CSV file
normalized_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN-Unseen data.csv'
data = pd.read_csv(normalized_file_path)

# Drop columns that should not be used for prediction (e.g., target labels)
data = data.drop(columns=['severity'], errors='ignore')

# Load the pre-trained model pipeline
pipeline = joblib.load('/content/drive/MyDrive/Research_Dysgraphia/svmfinal_pipeline_fold1.pkl')

# Attempt to extract feature names from the model pipeline, if possible
try:
    expected_features = pipeline.feature_names_in_
except AttributeError:
    # If feature names are not available, fallback to using the column names of the unseen data
    expected_features = data.columns

# Strip any leading/trailing spaces from column names in the unseen data
data.columns = data.columns.str.strip()

# Ensure unseen data matches the training feature names exactly
data = data.reindex(columns=expected_features, fill_value=0)

# Make predictions using the loaded pipeline
predictions = pipeline.predict(data)

# Print the prediction results
print("Predictions:", predictions)


Predictions: ['High']
